<a href="https://colab.research.google.com/github/ryoma150520/TF-IDF/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- ステップ1: Python 3.10の準備 ---
!sudo apt-get update -y
!sudo apt-get install -y python3.10 python3.10-distutils
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
print("--- Python 3.10の準備完了 ---")

# --- ステップ2: MeCab本体と関連ライブラリのインストール ---
!sudo apt-get install -y mecab libmecab-dev file
!python3.10 -m pip install mecab-python3
print("--- MeCab本体のインストール完了 ---")

# --- ステップ3: mecab-ipadic-neologd のインストール ---
# GitHubからソースコードをクローン（ダウンロード）
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
# ダウンロードしたディレクトリに移動して、インストールスクリプトを実行
!echo "yes" | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

print("\n--- mecab-ipadic-neologdのインストールが完了しました ---")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,067 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,587 kB]
Hit:13 https://ppa.launchpadconten

In [5]:
#ここは自分が使うものだけインストールすればいい
!python3.10 -m pip install google-cloud-translate pandas openpyxl wikiextractor
print("\n--- 追加ライブラリのインストールが完了しました ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 156.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 164.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27/27 [google-cloud-translate]

--- 追加ライブラリのインストールが完了しました ---


In [ ]:
# 日本語版Wikipediaの最新ダンプをダウンロード
!wget https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2

In [ ]:
# --json オプションを追加して、出力がJSON形式になるように修正。JSON形式にしてるけど自分が使うツールに合わせて変更してもいいかも
!python3.10 -m wikiextractor.WikiExtractor jawiki-latest-pages-articles.xml.bz2 -o extracted_wiki -b 10M --processes 8 --json

In [ ]:
#もし自分が何かしらのデータ（文章とか、単語とかを集めたエクセルファイルで計算する場合）持ってるならそのデータファイルを読み込めるセル。データファイル読み込ませたかったら今S2_data.xlsxになってるところ自分のエクセルファイルに置き換えればいい（Google認証のところは全部削除）。
from google.colab import files
import os
import re
import json
from collections import Counter


print("Excelファイル (S2_data.xlsx) と Google Cloud認証キー (JSONファイル) をアップロードしてください。")
uploaded = files.upload()

# アップロードされたファイル名を取得
excel_file_name = "S2_data.xlsx"

#以下無視
gcp_key_file_name = ""
for fn in uploaded.keys():
  if fn.endswith('.json'):
    gcp_key_file_name = fn
    print(f"認証キー '{fn}' を認識しました。")

if not gcp_key_file_name:
    print("エラー: Google Cloudの認証キー(JSONファイル)が見つかりません。")

# 環境変数を設定してPythonから認証キーを使えるようにする
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = gcp_key_file_name

In [ ]:
#IDFライク単語辞書作成。作成された辞書json形式でGoogle Driveのマイドライブってとこに格納されるようにしてる。
%%writefile create_freq_dict.py

import os
import re
import json
from collections import Counter
import MeCab


print("Wikipediaの全テキストから単語頻度辞書を作成します...")

# NEologd辞書と、mecabrc設定ファイルのパスを明示的に指定します。
NEOLOGD_DIC_PATH = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
MECABRC_PATH = "/etc/mecabrc"

try:
    # 辞書パスと設定ファイルパスの両方を指定
    tagger_options = f"-d {NEOLOGD_DIC_PATH} -r {MECABRC_PATH}"
    wakati_tagger = MeCab.Tagger(f"-Owakati {tagger_options}")
    wakati_tagger.parse('') # 初期化チェック
    print("MeCab Tagger (NEologd) の初期化に成功しました。")
except Exception as e:
    print(f"MeCab Tagger (NEologd) の初期化に失敗しました。エラー: {e}")
    # フォールバックもrcfileを指定
    try:
        print("フォールバックとしてデフォルト辞書での初期化を試みます...")
        wakati_tagger = MeCab.Tagger(f"-Owakati -r {MECABRC_PATH}")
        wakati_tagger.parse('')
        print("デフォルト辞書での初期化に成功しました。")
    except Exception as e2:
        print(f"デフォルト辞書での初期化にも失敗しました。エラー: {e2}")
        raise RuntimeError("MeCabを正常に初期化できませんでした。")

# --- 以降の処理 ---
word_counter = Counter()
total_word_count = 0
input_dir = 'extracted_wiki'

file_paths = [os.path.join(root, file) for root, _, files in os.walk(input_dir) for file in files if file.startswith('wiki_')]
file_count = len(file_paths)

for i, filepath in enumerate(file_paths):
    print(f"  - 処理中: {filepath} ({i+1}/{file_count})")
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                article = json.loads(line)
                text = article.get('text', '')
                text = re.sub(r'<.*?>', '', text)
                text = re.sub(r'\[\[.*?\|(.*?)\]\]', r'\1', text)
                text = re.sub(r'\[\[(.*?)\]\]', r'\1', text)
                tokens = wakati_tagger.parse(text).strip().split()
                word_counter.update(tokens)
                total_word_count += len(tokens)
            except json.JSONDecodeError:
                continue

print("\n単語頻度辞書の作成が完了しました。")
print(f"ユニーク単語数: {len(word_counter)}")
print(f"総単語数: {total_word_count}")

# 結果をGoogleドライブに保存
output_freq_file = '/content/drive/MyDrive/wiki_freq_neologd.json'
with open(output_freq_file, 'w', encoding='utf-8') as f:
    json.dump({
        'freq_map': dict(word_counter),
        'total_words': total_word_count
    }, f, ensure_ascii=False, indent=2)

print(f"\n頻度データをあなたのGoogleドライブ ('{output_freq_file}') に保存しました。")
print("これ以降は、「ノートブック②：メイン分析用」に切り替えてください。")

In [2]:
%%writefile create_freq_dict.py

import os
import re
import json
from collections import Counter
import MeCab

print("Wikipediaの全テキストから【文書頻度(Document Frequency)】辞書を作成します...")

# NEologd辞書と、mecabrc設定ファイルのパスを明示的に指定します。
NEOLOGD_DIC_PATH = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
MECABRC_PATH = "/etc/mecabrc"

try:
    # 辞書パスと設定ファイルパスの両方を指定
    tagger_options = f"-d {NEOLOGD_DIC_PATH} -r {MECABRC_PATH}"
    wakati_tagger = MeCab.Tagger(f"-Owakati {tagger_options}")
    wakati_tagger.parse('') # 初期化チェック
    print("MeCab Tagger (NEologd) の初期化に成功しました。")
except Exception as e:
    print(f"MeCab Tagger (NEologd) の初期化に失敗しました。エラー: {e}")
    # フォールバックもrcfileを指定
    try:
        print("フォールバックとしてデフォルト辞書での初期化を試みます...")
        wakati_tagger = MeCab.Tagger(f"-Owakati -r {MECABRC_PATH}")
        wakati_tagger.parse('')
        print("デフォルト辞書での初期化に成功しました。")
    except Exception as e2:
        print(f"デフォルト辞書での初期化にも失敗しました。エラー: {e2}")
        raise RuntimeError("MeCabを正常に初期化できませんでした。")

# --- ★★★ 修正点1: 変数名を役割に合わせて変更 ★★★ ---
doc_freq_counter = Counter() # 単語が出現した「文書数」をカウントする
total_document_count = 0     # 処理した「文書数」をカウントする
input_dir = 'extracted_wiki'

file_paths = [os.path.join(root, file) for root, _, files in os.walk(input_dir) for file in files if file.startswith('wiki_')]
file_count = len(file_paths)

for i, filepath in enumerate(file_paths):
    print(f"  - 処理中: {filepath} ({i+1}/{file_count})")
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            # 各行が1つの文書（記事）に対応する
            try:
                article = json.loads(line)
                text = article.get('text', '')

                # 簡単なテキストクリーニング
                text = re.sub(r'<.*?>', '', text)
                text = re.sub(r'\[\[.*?\|(.*?)\]\]', r'\1', text)
                text = re.sub(r'\[\[(.*?)\]\]', r'\1', text)

                tokens = wakati_tagger.parse(text).strip().split()

                # --- ★★★ 修正点2: この文書のユニークな単語のみをカウント ★★★ ---
                unique_tokens = set(tokens)
                doc_freq_counter.update(unique_tokens)

                # --- ★★★ 修正点3: 文書数をカウントアップ ★★★ ---
                total_document_count += 1

            except json.JSONDecodeError:
                continue

print("\n文書頻度辞書の作成が完了しました。")
print(f"ユニーク単語数: {len(doc_freq_counter)}")
print(f"総文書数（記事数）: {total_document_count}")

# --- ★★★ 修正点4: 新しいファイル名とJSON構造で結果を保存 ★★★ ---
output_idf_file = '/content/drive/MyDrive/wiki_tfidf_data.json'
with open(output_idf_file, 'w', encoding='utf-8') as f:
    json.dump({
        'total_documents': total_document_count,
        'doc_freq_map': dict(doc_freq_counter)
    }, f, ensure_ascii=False, indent=2)

print(f"\n文書頻度データをあなたのGoogleドライブ ('{output_idf_file}') に保存しました。")
print("これ以降は、メインの分析スクリプト(create_json.py)もこの新しいJSONファイルを読むように修正する必要があります。")

Writing create_freq_dict.py


In [6]:
#前のセル実行
!python3.10 create_freq_dict.py

Wikipediaの全テキストから【文書頻度(Document Frequency)】辞書を作成します...
MeCab Tagger (NEologd) の初期化に成功しました。
  - 処理中: extracted_wiki/AH/wiki_28 (1/775)
  - 処理中: extracted_wiki/AH/wiki_71 (2/775)
  - 処理中: extracted_wiki/AH/wiki_65 (3/775)
  - 処理中: extracted_wiki/AH/wiki_69 (4/775)
  - 処理中: extracted_wiki/AH/wiki_62 (5/775)
  - 処理中: extracted_wiki/AH/wiki_02 (6/775)
  - 処理中: extracted_wiki/AH/wiki_44 (7/775)
  - 処理中: extracted_wiki/AH/wiki_00 (8/775)
  - 処理中: extracted_wiki/AH/wiki_73 (9/775)
  - 処理中: extracted_wiki/AH/wiki_53 (10/775)
  - 処理中: extracted_wiki/AH/wiki_47 (11/775)
  - 処理中: extracted_wiki/AH/wiki_37 (12/775)
  - 処理中: extracted_wiki/AH/wiki_08 (13/775)
  - 処理中: extracted_wiki/AH/wiki_56 (14/775)
  - 処理中: extracted_wiki/AH/wiki_54 (15/775)
  - 処理中: extracted_wiki/AH/wiki_24 (16/775)
  - 処理中: extracted_wiki/AH/wiki_61 (17/775)
  - 処理中: extracted_wiki/AH/wiki_41 (18/775)
  - 処理中: extracted_wiki/AH/wiki_70 (19/775)
  - 処理中: extracted_wiki/AH/wiki_10 (20/775)
  - 処理中: extracted_wiki/AH/wiki_38 (21

In [ ]:
#ここで自分が用意したデータファイルの単語とTF-IDFの計算してる。英訳機能とかスコアのパーセンテージ化機能（Softmax）とか含まれてるからそれらに関するところは全部削除で大丈夫（無視って書いてるとこ削除、他もいらんとこあるかも。削除したらAIにエラー起きんように全体修正してもらえばいい）。品詞の設定も自分が用いる品詞に適宜変更して。
#もしかしたらここからは自分のVsCord開いてやったほうがいいかも（俺TF-IDFの計算以外もここでやってるから複雑になってる。エラーめっちゃ起きるかも）。多分前のセルで作ったjsonファイルchatgptかなんかにアップロードしてこういう風にTF-IDFの計算したいって聞いてVsCordコピペして実行の方が早い気がする。
%%writefile create_json.py
import pandas as pd
import MeCab
from collections import Counter, defaultdict
import json
import os
import math
import time
from google.cloud import translate_v2 as translate
import html # ★★★ HTMLデコードライブラリをインポート ★★★

# --- 設定項目 ---
# 1. MeCabの辞書パス
MECAB_DIC_PATH = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"

# 2. ジャンルIDマップ。無視
GENRE_ID_MAP = {
    "文芸・評論": "bungei", "ノンフィクション": "nonfiction", "ビジネス・経済": "business",
    "歴史・地理": "rekishi", "政治・社会": "politics", "芸能・エンターテインメント": "entertainment",
    "アート・建築・デザイン": "art", "人文・思想・宗教": "humanities", "暮らし・健康・料理": "life",
    "サイエンス・テクノロジー": "science", "趣味・実用": "hobby", "教育・自己啓発": "education",
    "スポーツ・アウトドア": "sports", "事典・年鑑・本・ことば": "dictionary", "音楽": "music",
    "旅行・紀行": "ryokou", "絵本・児童書": "kids", "コミックス": "comics"
}

# 3. Google CloudプロジェクトID。無視
GCP_PROJECT_ID = "my-translation-app-463407"  # ご自身のIDに設定してください

# 4. 文の翻訳を有効にするか。無視
TRANSLATE_SENTENCES = True

# 5. 読み込む頻度データファイル。さっき作ったファイル。同じディレクトリに移動させて。それか/content/drive/MyDrive/wiki_freq_neologd.jsonに置き換える。
WIKI_FREQ_FILE = "wiki_freq_neologd.json"

# 6. ファイル設定
EXCEL_FILE = "S2_data.xlsx"
SHEET_NAMES = list(GENRE_ID_MAP.keys())
OUTPUT_JSON_FILE = "analysis_data.json"
ROW_START, ROW_END = 1, 11

# 7. Softmax温度設定。無視
SOFTMAX_TEMPERATURE = 15.0

# --- クライアントとTaggerの初期化 ---
try:
    translate_client = translate.Client()
except Exception as e:
    print(f"Google翻訳クライアントの初期化に失敗。認証情報を確認してください。エラー: {e}")
    exit()

try:
    tagger_options = f'-d "{MECAB_DIC_PATH}"'
    tagger = MeCab.Tagger(tagger_options)
    wakati_tagger = MeCab.Tagger(f"-Owakati {tagger_options}")
    tagger.parse('')
    print("MeCab Tagger (NEologd) の初期化に成功しました。")
except RuntimeError as e:
    print(f"MeCabの初期化に失敗しました。: {e}")
    print(f"MECAB_DIC_PATH '{MECAB_DIC_PATH}' が正しいか確認してください。")
    exit()

# --- 関数定義 ---
def load_freq_data(filepath):
    print(f"ステップ1/5: 外部単語頻度データ '{filepath}' を読み込み中...")
    if not os.path.exists(filepath):
        print(f"  - エラー: ファイル '{filepath}' が見つかりません。")
        exit()
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if 'freq_map' not in data or 'total_words' not in data:
        print(f"  - エラー: '{filepath}' は 'freq_map' と 'total_words' のキーを持つ必要があります。")
        exit()
    print(f"  - 完了。{len(data['freq_map'])}語のデータを読み込みました。")
    return data['freq_map'], data['total_words']

#無視
def translate_texts(texts, target_language="en"):
    """テキストのリストを翻訳し、HTMLエンティティをデコードする"""
    if not texts: return {}
    unique_texts = list(set(text for text in texts if text and isinstance(text, str)))
    if not unique_texts: return {text: "" for text in texts}

    BATCH_SIZE, all_translations = 128, {}
    print(f"  - 合計 {len(unique_texts)}件のユニークなテキストを翻訳します...")
    for i in range(0, len(unique_texts), BATCH_SIZE):
        chunk = unique_texts[i:i + BATCH_SIZE]
        print(f"    - 翻訳中... (チャンク {i//BATCH_SIZE + 1} / {math.ceil(len(unique_texts)/BATCH_SIZE)})")
        try:
            results = translate_client.translate(chunk, target_language=target_language)
            # ★★★ HTMLエンティティをデコードする処理を追加 ★★★
            for item in results:
                decoded_text = html.unescape(item['translatedText'])
                all_translations[item['input']] = decoded_text
        except Exception as e:
            print(f"      - 翻訳API呼び出し中にエラー: {e}")
            for text in chunk: all_translations[text] = ""
        time.sleep(0.1)
    return all_translations

#無視
def apply_softmax(word_objects, temperature=1.0):
    if not word_objects: return []
    if temperature <= 0: temperature = 1.0
    scores = [item['score'] / temperature for item in word_objects]
    max_score = max(scores)
    exps = [math.exp(s - max_score) for s in scores]
    sum_exps = sum(exps)
    if sum_exps == 0: return word_objects
    for i, item in enumerate(word_objects):
        item['score'] = exps[i] / sum_exps
    return word_objects

def get_tokens_from_text(text):
    if not isinstance(text, str): text = str(text)
    return wakati_tagger.parse(text).strip().split()

def analyze_pos_and_score(text, freq_map, total_corpus_words):
    if not isinstance(text, str): text = str(text)

    tokens_for_tf = get_tokens_from_text(text)
    tf = Counter(tokens_for_tf)
    node = tagger.parseToNode(text)
    results = defaultdict(list)

    max_idf = math.log((total_corpus_words / 1) + 1)

    while node:
        surface = node.surface
        features = node.feature.split(',')
        if not surface or len(features) < 2:
            node = node.next
            continue

        freq_in_doc = tf.get(surface, 0)
        if freq_in_doc == 0:
            node = node.next
            continue

        freq_in_corpus = freq_map.get(surface)
        idf = max_idf if freq_in_corpus is None else math.log((total_corpus_words / (freq_in_corpus + 1)) + 1)

        score = (freq_in_doc * idf) + math.log(freq_in_doc + 1)
        word_obj = {"word": surface, "score": score}

        pos, sub_pos1 = features[0], features[1]

        if pos == "名詞":
            if sub_pos1 in ["代名詞", "非自立", "数", "接尾", "副詞可能", "形容動詞語幹"]:
                results["nouns_functional"].append(word_obj)
            else:
                results["nouns_other"].append(word_obj)
        elif pos == "動詞":
            results["verbs"].append(word_obj)
        elif pos == "形容詞":
            results["adjectives"].append(word_obj)
        elif pos == "形容動詞":
            results["adjectival_verbs"].append(word_obj)

        node = node.next
    return results

#無視
def extract_important_words(analysis_results_raw):
    important = defaultdict(list)
    for pos_key, words in analysis_results_raw.items():
        word_counts = Counter(item['word'] for item in words)
        important_word_names = {word for word, count in word_counts.items() if count >= 2}

        important_words = [item for item in words if item['word'] in important_word_names]
        important[pos_key] = important_words
    return important

#無視
def extract_important_sentences_by_score(text, word_score_map):
    if not isinstance(text, str): return "説明文がテキスト形式ではありません"

    sentences = [s.strip() for s in text.split("。") if s.strip()]
    if not sentences: return "該当する文がありません"

    scored_sentences = []
    for sentence in sentences:
        tokens = get_tokens_from_text(sentence)
        sentence_score = sum(word_score_map.get(token, 0) for token in tokens)
        scored_sentences.append((sentence, sentence_score))

    scored_sentences = [s for s in scored_sentences if s[1] > 0]
    if not scored_sentences: return "重要度の高い単語を含む文がありません"

    scored_sentences.sort(key=lambda x: x[1], reverse=True)
    return scored_sentences[0][0] + "。"

# --- メイン処理 ---
def main():
    freq_map, total_words = load_freq_data(WIKI_FREQ_FILE)

    print("ステップ2/5: 全てのドキュメントをExcelから読み込み中...")
    all_docs_raw = []
    for sheet_name in SHEET_NAMES:
        try:
            df = pd.read_excel(EXCEL_FILE, sheet_name=sheet_name, usecols=[1, 3], header=None, engine="openpyxl")
            for i in range(ROW_START, ROW_END):
                if i < len(df):
                    all_docs_raw.append({
                        "sheet_name": sheet_name,
                        "title": str(df.iloc[i, 0]),
                        "text": str(df.iloc[i, 1])
                    })
        except Exception as e:
            print(f"  - シート '{sheet_name}' の読み込みエラー: {e}")

    print("ステップ3/5: 各ドキュメントを解析し、スコアを計算中...")
    all_docs_processed = []
    for i, doc in enumerate(all_docs_raw):
        analysis_results_raw = analyze_pos_and_score(doc['text'], freq_map, total_words)

        aggregated_analysis = defaultdict(lambda: defaultdict(float))
        word_score_map = {}
        for category, word_list in analysis_results_raw.items():
            for item in word_list:
                aggregated_analysis[category][item['word']] += item['score']
        for category_scores in aggregated_analysis.values():
            for word, score in category_scores.items():
                word_score_map[word] = score

        final_analysis_results = {
            cat: apply_softmax([{"word": w, "score": s} for w, s in scores.items()], temperature=SOFTMAX_TEMPERATURE)
            for cat, scores in aggregated_analysis.items()
        }

        important_words_raw = extract_important_words(analysis_results_raw)
        aggregated_important = defaultdict(lambda: defaultdict(float))
        for category, word_list in important_words_raw.items():
            for item in word_list:
                aggregated_important[category][item['word']] += item['score']

        final_important_words = {
            cat: apply_softmax([{"word": w, "score": s} for w, s in scores.items()], temperature=SOFTMAX_TEMPERATURE)
            for cat, scores in aggregated_important.items()
        }

        doc.update({
            "genre_id": GENRE_ID_MAP.get(doc['sheet_name'], doc['sheet_name'].lower()),
            "title_id": f"title{(i % (ROW_END - ROW_START)) + 1:02d}",
            "important_sentence": extract_important_sentences_by_score(doc['text'], word_score_map),
            "analysis": final_analysis_results,
            "important_words": final_important_words
        })
        all_docs_processed.append(doc)

#無視
    print("ステップ4/5: テキストを翻訳中...")
    words_to_translate, sentences_to_translate = set(), []
    for doc in all_docs_processed:
        if TRANSLATE_SENTENCES:
            sentences_to_translate.append(doc['text'])
            sentences_to_translate.append(doc['important_sentence'])
        for category in doc['analysis']:
            for item in doc['analysis'][category]:
                words_to_translate.add(item['word'])
        for category in doc['important_words']:
            for item in doc['important_words'][category]:
                words_to_translate.add(item['word'])

    word_translation_map = translate_texts(list(words_to_translate))
    sentence_translation_map = translate_texts(sentences_to_translate) if TRANSLATE_SENTENCES else {}

    print("ステップ5/5: 最終的なJSONデータを構築中...")
    final_data = defaultdict(list)
    for doc in all_docs_processed:
        for category in doc['analysis']:
            for item in doc['analysis'][category]:
                item['translation'] = word_translation_map.get(item['word'], "")
        for category in doc['important_words']:
            for item in doc['important_words'][category]:
                item['translation'] = word_translation_map.get(item['word'], "")

        doc['text_en'] = sentence_translation_map.get(doc['text'], "")
        doc['important_sentence_en'] = sentence_translation_map.get(doc['important_sentence'], "")

        doc.pop('sheet_name', None)
        final_data[doc['genre_id']].append(doc)

    id_to_genre_map = {v: k for k, v in GENRE_ID_MAP.items()}
    final_json_data = {id_to_genre_map.get(gid, gid): docs for gid, docs in final_data.items()}

    with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_json_data, f, ensure_ascii=False, indent=2)

    print(f"\n処理完了。'{OUTPUT_JSON_FILE}' を保存しました。")

if __name__ == "__main__":
    main()

In [ ]:
#前のセル実行
!python3.10 create_json.py

In [ ]:
#これは無視
!cp -r extracted_wiki /content/drive/MyDrive/
!cp -r jawiki-latest-pages-articles.xml.bz2 /content/drive/MyDrive/